# Automatic Differentiation
Automatic differentiation (줄여서 AD)는 함수의 미분 값을 "자동"으로 계산해주는 알고리즘입니다.
역전파 (backpropagation) 알고리즘으로 알고 계실 수도 있을 것 같습니다.
하지만 둘은 다른 용어입니다.

## 두 가지 모드
AD는 크게 두 가지 방법이 있습니다.
Forward mode와 reverse mode입니다.
효율적인 계산을 위해서 각 mode가 어떻게 동작하는지 알고, 상황에 따라 적절한 알고리즘을 선택해서 사용할 필요가 있습니다.

결론부터 말하자면,
함수
\begin{equation*}
    f: \mathbb{R}^{d_\mathrm{in}} \rightarrow \mathbb{R}^{d_\mathrm{out}}
\end{equation*}
가 있을 때
{math}`d_\mathrm{in} < d_\mathrm{out}`인 경우 forward mode가 빠르고,
반대의 경우 {math}`d_\mathrm{in} > d_\mathrm{out}`에 reverse mode가 빠릅니다.

Deep Learning의 경우 손실 함수 (Loss function) $L$의 input은 neural network의 parameter $\theta \in \mathbb{R}^p$가 되고, output은 손실 함수의 값 $L(\theta) \in \mathbb{R}$이 됩니다.
많은 경우에 $p \gg 1$ 이므로 reverse mode가 빠릅니다.

실험을 통해 보겠습니다.

In [ ]:
import jax  # AD library JAX를 import 합니다.
import jax.numpy as jnp  # jnp는 numpy와 "거의" 같습니다.


def f(x: float):
    # scalar를 받아서 [1, x, ..., x^9999]를 리턴하는 함수, d_in < d_out
    return jnp.power(x, jnp.arange(10000))


def g(y: jnp.ndarray):
    # array를 받아서 제곱한 후 평균을 취하는 함수, d_in > d_out
    return (y**2).mean()


f_fwd = jax.jacfwd(f)
f_rev = jax.jacrev(f)
g_fwd = jax.jacfwd(g)
g_rev = jax.jacrev(g)

x = 1.0
y = jnp.ones((10000,))

함수 `f`의 경우 $d_\mathrm{in} = 1 \ll d_\mathrm{out} = 10^4$ 이므로, forward mode로 미분한 함수 `f_fwd`가 reverse mode로 미분한 함수 `g_rev`보다 빨라야 합니다.

In [2]:
%timeit f_fwd(x).block_until_ready()

4.55 ms ± 599 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%timeit f_rev(x).block_until_ready()

10.8 ms ± 338 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


`f_fwd`가 약 두배 정도 빠른 것을 볼 수 있습니다.

반면, 함수 `g`의 경우 $d_\mathrm{in} = 10^4 \gg d_\mathrm{out} = 1$ 이므로, reverse mode로 미분한 함수 `g_rev`가 forward mode로 미분한 함수 `g_fwd` 보다 빨라야 합니다.

In [4]:
%timeit g_fwd(y).block_until_ready()

11.9 ms ± 101 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit g_rev(y).block_until_ready()

6.52 ms ± 452 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


`g_rev`가 약 두배 정도 빠른 것을 볼 수 있습니다.

